In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [3]:
# # Define the path to your dataset folder
# data_path = "/"

# Load datasets directly from Google Drive
import pandas as pd

train_df = pd.read_csv("train_data.csv")
val_df = pd.read_csv("val_data.csv")
test_df = pd.read_csv("test_data.csv")


In [ ]:
!pip uninstall -y wandb



In [5]:
import pandas as pd

df = pd.read_csv('train_data.csv')
print(df.head())


                English             Italian
0         MEMBRANE UD-1       MEMBRANA UD-1
1      PLASTIC BASE LT8   BASE CABINATO LT8
2      COMMUTATOR 6 POS   COMMUTATORE 6 POS
3     MOTOR CASING K120  CARTER MOTORE K120
4  SCREW F3,5X13 N11725                VITE


In [6]:
!pip install transformers datasets sentencepiece sacrebleu evaluate accelerate

In [10]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict


# # Load the datasets
# train_df = pd.read_csv('/content/drive/MyDrive/translation_data/train_data.csv')
# val_df = pd.read_csv('/content/drive/MyDrive/translation_data/val_data.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/translation_data/test_data.csv')

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Reduce dataset
train_dataset = train_dataset.select(range(min(300, len(train_dataset))))
val_dataset = val_dataset.select(range(min(50, len(val_dataset))))
test_dataset = test_dataset.select(range(min(50, len(test_dataset))))

# Verify the new sizes
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")




Train size: 300
Validation size: 50
Test size: 50


In [13]:
!pip install --upgrade transformers


In [16]:
from transformers import MBart50TokenizerFast
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint, cache_dir='./cache', force_download=True)


OSError: Can't load tokenizer for 'facebook/mbart-large-50-many-to-many-mmt'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/mbart-large-50-many-to-many-mmt' is the correct path to a directory containing all relevant files for a MBart50TokenizerFast tokenizer.

In [14]:

from transformers import MBart50TokenizerFast

model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="it_IT")

max_length = 128

def tokenize_data(example):
    inputs = tokenizer(example['English'], max_length=max_length, truncation=True, padding="max_length")
    targets = tokenizer(example['Italian'], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])
tokenized_val = val_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])
tokenized_test = test_dataset.map(tokenize_data, batched=True, remove_columns=['English', 'Italian'])



OSError: Can't load tokenizer for 'facebook/mbart-large-50-many-to-many-mmt'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/mbart-large-50-many-to-many-mmt' is the correct path to a directory containing all relevant files for a MBart50TokenizerFast tokenizer.

# Setup Data Collator and Metrics

In [ ]:
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_checkpoint, padding=True)

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    return {"bleu": result["score"]}


#Initialize the mBART Model

In [ ]:
from transformers import MBartForConditionalGeneration

model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)


#Define Training Arguments

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
       output_dir="mbart-eng-it-domain-specific",
       eval_strategy="epoch",
       save_strategy="epoch",
       learning_rate=2e-5,
       per_device_train_batch_size=4,  #  batch size
       per_device_eval_batch_size=4,  #  batch size
       num_train_epochs=3,
       weight_decay=0.01,
       predict_with_generate=True,
       save_total_limit=2,
       fp16=True,
       report_to=None
   )

# Initialize the Trainer

In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-9-cd3df47410e1>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


#Start training

In [ ]:
trainer.train()


# Disable wandb to train free

#Evaluate the model

In [ ]:
eval_results = trainer.evaluate(tokenized_test)
print(f"BLEU score on test set: {eval_results['eval_bleu']:.2f}")


NameError: name 'trainer' is not defined

# Save model and tokenizer

In [ ]:
trainer.save_model("mbart-eng-it-trained")
tokenizer.save_pretrained("mbart-eng-it-trained")


#Inference - Testing translations

In [ ]:
# from transformers import pipeline

# translator = pipeline(
#     "translation",
#     model="mbart-eng-it-trained",
#     tokenizer="mbart-eng-it-trained",
#     src_lang="en_XX",
#     tgt_lang="it_IT"
# )

# sentence = "This is a domain-specific sentence for translation."
# translated_sentence = translator(sentence)[0]['translation_text']
# print(f"Translated sentence: {translated_sentence}")
